# 분석 프로그래밍 팀 프로젝트 코드
















# 데이터 불러오기 및 전처리

In [ ]:
import pandas as pd #판다스 패키지
import numpy as np #넘파이 패키지

import seaborn as sns #시각화 패키지
import matplotlib.pylab as plt #시각화 패키지
from matplotlib import cm #시각화 패키지에서 컬러 조정

import sys
print(sys.version)

from scipy import stats
from scipy.stats import trim_mean
from statsmodels import robust #머신러닝 패키지

from sklearn.cluster import KMeans  # k-means 군집분석
from sklearn.metrics import silhouette_score  # 군집분석 평가를 위한 silhouette 값 계산
from IPython.display import Image #이미지 넣는 모듈

from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name) #한글안나올때 넣는 모듈
plt.rcParams['axes.unicode_minus'] = False # 폰트 안나올때
# plt.rc('font', family='AppleGothic') # for MAC OS users
plt.rc('font', family=font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()) # for Windows OS user

In [ ]:
tr = pd.read_csv(r"구매내역정보.csv", encoding='cp949')
cs = pd.read_csv(r"Demo.csv", encoding='cp949')
df = pd.merge(tr, cs, on = "ID")
display(df.head())

In [ ]:
# 거주지역 분할
df['거주지역_광역'] = df['거주지역'].apply(lambda x : x.split(' ')[0])
df['거주지역_기초'] = df['거주지역'].apply(lambda x : x.split(' ')[1])

# 구매지역 분할
df['구매지역_광역'] = df['구매지역'].apply(lambda x : x.split(' ')[0])
df['구매지역_기초'] = df['구매지역'].apply(lambda x : x.split(' ')[1])

# 연령대 분할후
df['연령대'] = df['연령'].apply(lambda x : '50대' if x <60 else '60대' if x <70 else '70대' if x <80 else '80대')

# 성별 구분
df['성별'] = df['성별'].apply(lambda x : '남자' if x ==1 else '여자')

# 구매일자 구분
df['구매일자'] = df['구매일자'].astype(str).astype('datetime64') # 날짜정보를 담고 있는 de_dt 컬럼의 데이터 유형이 정수이기 때문
df['연'] = df['구매일자'].dt.year
df['월'] = df['구매일자'].dt.month
df["요일"] = df["구매일자"].apply(lambda x: ('월','화','수','목','금','토','일')[x.weekday()] + '요일')

# 판매단가 컬럼 생성
df['판매단가'] = (df['구매금액']/df['구매수량']).astype('int')

display(df.head())

# RFM 분석

### frequency

In [ ]:
frequency = df.groupby("ID")["구매일자"].nunique().sort_values().reset_index()
frequency

### monetary

In [ ]:
from scipy.stats import norm

monetary = df.groupby("ID")["구매금액"].nunique().sort_values().reset_index()
monetary.구매금액.apply(lambda x: np.log1p(x))
monetary

### recency

In [ ]:
now = df.구매일자.max()+pd.DateOffset(days=1)
recency = (df.groupby('ID')['구매일자']
           .agg(lambda x,y: (y - x).min().days, now).reset_index()
           .rename(columns={'구매일자':'recency'})
          )

recency["recency"] = 365 - recency["recency"] 
recency

## rfm score

In [ ]:
rfm = pd.merge(frequency, monetary, on = "ID").merge(recency, on = "ID").set_index("ID")
rfm = (rfm/(rfm.max()))
rfm =rfm.rename(columns = {"구매일자":"frequency", "구매금액":"monetary"})

In [ ]:
rfm["score"] = rfm["frequency"] * rfm["recency"] * rfm["monetary"]
rfm = rfm.reset_index()

In [ ]:
rfm["등급"] = 0
for i in range(0, len(rfm["score"])):
    if rfm["score"][i] <= rfm["score"].quantile(1/3):
        rfm["등급"][i] = "3등급"
    elif rfm["score"][i] <= rfm["score"].quantile(2/3):
        rfm["등급"][i] = "2등급"
    else:
        rfm["등급"][i] = "1등급"
        
rfm = rfm[["ID", "등급"]]

# RFM등급별 대분류제품군별 총구매액

In [ ]:
df = df.merge(rfm, on = "ID")

In [ ]:
aa = pd.pivot_table(df, index = "등급", columns = "상품대분류명",
                values = "구매금액", aggfunc = "sum", fill_value = 0).plot(marker = "o", markersize = 10, figsize = (20, 10))
plt.title("등급별 상품대분류명별 총구매액", fontsize = 20)
plt.show()

## RFM등급별 상품중분류별 총구매액

In [ ]:
fig, axes = plt.subplots(4, 2, figsize = (20, 30))


pd.pivot_table(df.query("상품대분류명 == '가전제품'"), index = "등급", 
               columns = "상품중분류명", values = "구매금액", aggfunc = "sum").plot(marker = "o", markersize = 10, ax = axes[0, 0])
axes[0, 0].set_title("가전제품 중분류별 총구매액", fontsize = 15)
axes[0, 0].set_ylabel("제품별 총구매액")
axes[0, 0].legend(loc = "upper right")

pd.pivot_table(df.query("상품대분류명 == '의류잡화'"), index = "등급", 
               columns = "상품중분류명", values = "구매금액", aggfunc = "sum").plot(marker = "o", markersize = 10, ax = axes[0, 1])
axes[0, 1].set_title("의류잡화 중분류별 총구매액", fontsize = 15)
axes[0, 1].set_ylabel("제품별 총구매액")
axes[0, 1].legend(loc = "upper right")

pd.pivot_table(df.query("상품대분류명 == '명품'"), index = "등급", 
               columns = "상품중분류명", values = "구매금액", aggfunc = "sum").plot(marker = "o", markersize = 10, ax = axes[1, 0])
axes[1, 0].set_title("명품 총구매액", fontsize = 15)
axes[1, 0].set_ylabel("제품별 총구매액")
axes[1, 0].legend(loc = "upper right")

pd.pivot_table(df.query("상품대분류명 == '레포츠'"), index = "등급", 
               columns = "상품중분류명", values = "구매금액", aggfunc = "sum").plot(marker = "o", markersize = 10, ax = axes[1, 1])
axes[1, 1].set_title("레포츠 중분류별 총구매액", fontsize = 15)
axes[1, 1].set_ylabel("제품별 총구매액")
axes[1, 1].legend(loc = "upper right")

pd.pivot_table(df.query("상품대분류명 == '식품'"), index = "등급", 
               columns = "상품중분류명", values = "구매금액", aggfunc = "sum").plot(marker = "o", markersize = 10, ax = axes[2, 0])
axes[2, 0].set_title("식품 중분류별 총구매액", fontsize = 15)
axes[2, 0].set_ylabel("제품별 총구매액")
axes[2, 0].legend(loc = "upper right")

pd.pivot_table(df.query("상품대분류명 == '가구'"), index = "등급", 
               columns = "상품중분류명", values = "구매금액", aggfunc = "sum").plot(marker = "o", markersize = 10, ax = axes[2, 1])
axes[2, 1].set_title("가구 총구매액", fontsize = 15)
axes[2, 1].set_ylabel("제품별 총구매액")
axes[2, 1].legend(loc = "upper right")

pd.pivot_table(df.query("상품대분류명 == '생활잡화'"), index = "등급", 
               columns = "상품중분류명", values = "구매금액", aggfunc = "sum").plot(marker = "o", markersize = 10, ax = axes[3, 0])
axes[3, 0].set_title("생활잡화 중분류별 총구매액", fontsize = 15)
axes[3, 0].set_ylabel("제품별 총구매액")
axes[3, 0].legend(loc = "upper right")

pd.pivot_table(df.query("상품대분류명 == '생활잡화'"), index = "등급", 
               columns = "상품중분류명", values = "구매금액", aggfunc = "sum").plot(marker = "o", markersize = 10, ax = axes[3, 0])
axes[3, 0].set_title("생활잡화 중분류별 총구매액", fontsize = 15)
axes[3, 0].set_ylabel("제품별 총구매액")
axes[3, 0].legend(loc = "upper right")

plt.show()

# 이상치 제거

In [ ]:
pd.pivot_table(df, index = "ID", columns = "상품대분류명", values = "구매금액", aggfunc = "sum", fill_value = 0).plot.box()
plt.title("ID별 제품군별 총구매액 이상치 확인")
plt.show()

In [ ]:
df.query("상품대분류명 == '레포츠'").groupby("ID")["구매금액"].sum().sort_values(ascending = False).index[0]

In [ ]:
#이상치 제거
idx = df[df["ID"]==700882892].index
df = df.drop(idx)

# 군집분석

### [Feature 1] 총구매액, 구매건수, 평균구매액, 최대구매액

In [ ]:
features = df.groupby('ID')['구매금액'].agg([
    ('총구매액',np.sum), 
    ('구매건수', np.size), 
    ('평균구매액', lambda x: np.round(np.mean(x))),
    ('최대구매액', np.max),
]).reset_index()
features

In [ ]:
features.iloc[:,1:].hist(bins=20, figsize=(15,10))
plt.show()

In [ ]:
df['구매일자']=df.구매일자.astype(str).astype('datetime64')

### [Feature 2] 내점일수, 구매주기, 주말방문율

In [ ]:
f = df.groupby("ID")['구매일자'].agg([('내점일수',lambda x: x.nunique()),('구매주기', lambda x : int(((x.max() - x.min()).days) / x.nunique())),('주말방문율', lambda x: (x.dt.weekday > 4).mean().round(2)) 
]).reset_index()
features = features.merge(f, how='left'); features

In [ ]:
sns.pairplot(data=f.iloc[:,1:], height=3)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18,6))

sns.histplot(data=f, x='내점일수', kde=True, ax=ax[0])
sns.histplot(data=f, x='구매주기', kde=True, ax=ax[1])
sns.histplot(data=f, x='주말방문율', kde=True, ax=ax[2])
plt.show()

### [Feature3] 거래 당 평균 구매액, 구매건수

In [ ]:
visit_cnt = df.groupby('ID')['구매일자'].agg(lambda x: x.nunique()) #롯데에서 구매한 사실 자체만 집중. 
per_amount = (df.groupby('ID')['구매금액'].sum() / visit_cnt).round(0).reset_index().rename(columns={0:'거래당구매액'})
per_count = (df.groupby('ID')['구매금액'].count() / visit_cnt).round(1).reset_index().rename(columns={0:'거래당구매건수'})

f= pd.merge(per_amount, per_count, on='ID')
features = features.merge(f, how='left'); features

### [Feature4]주구매시간

In [ ]:
pd.DataFrame(df.query("ID == '999375162'")["구매시간"]).value_counts()

In [ ]:
f = df.groupby('ID')['구매시간'].agg([('주구매시간', lambda x: int(x.value_counts().index[0]))]).reset_index()
features= features.merge(f, how="left"); features

In [ ]:
# 범주형 속성 값의 빈도 출력
fig, ax = plt.subplots(figsize=(8, 6))
sns.countplot(y='주구매시간', data=f, alpha=0.5)  

### [Feature5]가격선호도

In [ ]:
# 판매단가 계산
df['price'] = df['구매금액'] / df['구매수량']

# 상품중분류별 평균 판매단가에 대한 4분위수 계산 => 3Q 이상을 고가상품으로 설정
price_4q = df.groupby('상품중분류명')['price'].mean().quantile([.25,.5,.75])
pd.DataFrame(price_4q)

In [ ]:
f = (df.groupby('ID')['price']
     .agg([('고가상품구매율', lambda x: (x > price_4q.iloc[2]).mean().round(2))])
     .reset_index())
features = features.merge(f, how='left')
features.sort_values(by=['고가상품구매율'], ascending=False)

### [Feature6]구매추세

In [ ]:
# 마지막주부터 10주전까지 (총 10주 간) 매주 구매건수를 계산하여 리스트에 넣음
week_to = df.구매일자.max()
week_trans = []
for i in range(10):
    week_from = week_to + pd.DateOffset(weeks=-1)
    week_trans.append(df.query('@week_from < 구매일자 <= @week_to')
                      .groupby('ID')['구매금액']
                      .agg([(f'w{10-i}', 'size')])
                      .reset_index())
    week_to = week_from

# 리스트로부터 데이터프레임 변환    
f = pd.DataFrame({'ID': df.ID.unique()})
for w in week_trans[::-1]:
    f = pd.merge(f, w, how='left')
f = f.fillna(0)

f['구매추세'] = f.apply(lambda x: np.polyfit(range(10), x[1:], 1)[0].round(2), axis=1)
features = features.merge(f.iloc[:,[0,-1]], how='left'); features

In [ ]:
sns.kdeplot(f.구매추세, color='m')
plt.show()

### [Feature 7]취소건수

In [ ]:
cancel_df = pd.DataFrame(df.groupby("ID")["취소여부"].agg([("취소건수",sum)]).reset_index())
features = pd.merge(features,cancel_df,how='left')
features

### [Feature 8]액티브시니어 특징이 드러나는 상품중분류별 구매금액

In [ ]:
a=df.query("상품중분류명 == '휴대폰/태블릿'").groupby("ID")["구매금액"].agg([("휴대폰/태블릿구매금액",sum)]).reset_index()
b=df.query("상품중분류명 == '화장품'").groupby("ID")["구매금액"].agg([("화장품구매금액",sum)]).reset_index()
c=df.query("상품대분류명 == '레포츠'").groupby("ID")["구매금액"].agg([("레포츠구매금액",sum)]).reset_index()
d=df.query("상품중분류명 == '여성용의류-SPA'").groupby("ID")["구매금액"].agg([("여성용의류-SPA구매금액",sum)]).reset_index()
e=df.query("상품중분류명 == '명품'").groupby("ID")["구매금액"].agg([("명품구매금액",sum)]).reset_index()
f=df.query("상품중분류명 == '가공식품'").groupby("ID")["구매금액"].agg([("가공식품구매금액",sum)]).reset_index()
g=df.query("상품대분류명 == '가전제품'").groupby("ID")["구매금액"].agg([("가전제품구매금액",sum)]).reset_index()
features = pd.merge(features, a, how='left')
features = pd.merge(features, b, how='left')
features = pd.merge(features, c, how='left')
features = pd.merge(features, d, how='left')
features = pd.merge(features, e, how='left')
features = pd.merge(features, f, how='left')
features = pd.merge(features, g, how='left')

features = features.fillna(0)
features



### 상관계수(-1 ~ +1)를 통해 feature 간의 유사성을 파악

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(features.iloc[:,1:].corr(), annot=True, cmap="YlGnBu")
plt.show()

In [ ]:
# 양의 상관관계가 매우 높음.
sns.lmplot(x='구매건수', y='내점일수', data=features, 
           line_kws={'color':"green"}, scatter_kws={'edgecolor':"white"})
plt.show()

### 불필요한 feature 삭제

In [ ]:
features = features.drop(columns='평균구매액')
features = features.drop(columns='구매건수')

### 표준화

In [ ]:
X = features.iloc[:,1:].apply(lambda x: (x - x.mean()) / x.std()) 
X

## 최적 군집 수(K) 도출

### 군집 수에 따른 SSE(군집내 오차제곱합) 계산

In [ ]:
wss = []
sis = []
for k in range(2,10):
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(X)
    wss = np.append(wss, kmeans.inertia_)

### 군집 수에 따른 SSE 시각화

In [ ]:
fig, ax = plt.subplots()
line = ax.plot(np.arange(2,10), wss, 'go--', label='SSE')
ax.set_ylim(wss.min()*0.55, wss.max()*1.05)
ax.set_xlabel('군집 수')
ax.set_ylabel('SSE')
labels = [l.get_label() for l in line]
plt.legend(line, labels)
plt.show()

In [ ]:
#wss비교
wss
for i in range(0, 7):
    print(wss[i] - wss[i+1])

In [ ]:
best_k = 3

In [ ]:
kmeans = KMeans(n_clusters=best_k, random_state=0)
kmeans.fit(X)

In [ ]:
features['세그먼트'] = kmeans.labels_
features

### 군집분석을 통한 고객 세분화

In [ ]:
plt.figure(figsize=(6,6))
features.세그먼트.value_counts().plot.pie(autopct='%.2f%%', figsize=(7,7), 
                                      startangle=30, legend=False)
plt.title("군집 비율")
plt.axis('equal')
plt.show()

In [ ]:
fig, ax = plt.subplots(best_k, 1, figsize=(8,5), sharex=True, sharey=True)
ax = ax.ravel()
plt.tight_layout() # 자동으로 레이아웃 조정

for i in range(kmeans.n_clusters):
    ax[i].set_title('cluster %d' % i)
    sns.barplot(x=np.arange(X.columns.size), y=kmeans.cluster_centers_[i], ax=ax[i])
    ax[i].set_xticks(np.arange(X.columns.size))
    ax[i].set_xticklabels(X.columns, rotation=90)

# 군집2의 우수성

In [ ]:
df = df.merge(features[["ID", "세그먼트"]], on = "ID")

In [ ]:
#세그먼트 비율
df.groupby("세그먼트")["ID"].nunique().plot.pie(startangle = 90, explode = [0, 0, 0.1], 
                                            autopct = "%1.1f%%", figsize = (8, 8), colors =  ['silver', 'whitesmoke', 'skyblue'])
plt.ylabel("")
plt.title("군집 별 인원비율", fontsize = 15)
plt.show()

In [ ]:
#군집 별 구매액비율
df.groupby("세그먼트")["구매금액"].sum().plot.pie(startangle = 90, explode = [0, 0, 0.1],
                                          autopct = "%1.1f%%", figsize = (8, 8), colors =  ['silver', 'whitesmoke', 'skyblue'])
plt.ylabel("")
plt.title("군집 별 총구매액", fontsize = 15)
plt.show()

In [ ]:
#군집 별 구매액비율
df.groupby("세그먼트")["구매수량"].count().plot.pie(startangle = 90, explode = [0, 0, 0.1],
                                            autopct = "%1.1f%%", figsize = (8, 8), colors =  ['silver', 'whitesmoke', 'skyblue'])
plt.ylabel("")
plt.title("군집 별 거래비중", fontsize = 15)
plt.show()

In [ ]:
df.query("세그먼트 == '2'").등급.value_counts().plot.pie(figsize = (8, 8), autopct = "%1.1f%%", colors = ["skyblue", "gray"])
plt.title("군집2의 rfm등급 비율")
plt.ylabel("")
plt.show()

# 타겟시장 구매행태 데이터 분석

In [ ]:
vip = df.query("세그먼트 == '2'")

In [ ]:
# 군집별 가전제품 총구매액
pd.pivot_table(df.query("상품대분류명 == '가전제품'"), index = "세그먼트", 
               columns = "상품중분류명", values = "구매금액", aggfunc = "sum").plot(marker = "o", markersize = 6)
plt.title("가전제품 중분류별 총구매액", fontsize = 15)
plt.ylabel("제품별 총구매액")
plt.legend(loc = "upper right",fontsize=8)
plt.show()

In [ ]:
df.query('세그먼트==2').groupby('상품중분류명')['구매금액'].sum().plot(kind='bar', figsize = (8, 6))
plt.title('상품중분류별 구매금액(세그먼트2)')
plt.show()

In [ ]:
# 요일별 제품군별 총구매액
a = pd.pivot_table(vip, index = "상품중분류명", columns = "요일", values = "구매금액", aggfunc = "sum", fill_value = 0).T
a = a.reindex(['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일'])
plt.figure(figsize=(10,8))
sns.heatmap(a.T)
plt.title("제품별 요일별 총구매액")
plt.show()

In [ ]:
# 제품별 요일별 총구매액
b = pd.pivot_table(vip, index = "상품중분류명", columns = "구매시간", values = "구매금액", aggfunc = "sum", fill_value = 0)
plt.figure(figsize=(10,8))
sns.heatmap(b)
plt.title("제품별 요일별 총구매액")
plt.show()

In [ ]:
# 월별 레포츠 상품의 총구매액 비교

pd.pivot_table(vip.query("상품중분류명 == '골프용품/의류'"), index = "월", columns = "성별", 
               values = "구매금액", aggfunc = "sum", fill_value = 0).plot.bar(figsize = (8, 6), rot = 0)
plt.title("골프용품/의류 월별 총구매액", fontsize = 15)
plt.ylabel("구매액")
plt.xlabel("월")
plt.show()

In [ ]:
# 월별 레포츠 상품의 총구매액 비교

pd.pivot_table(vip.query("상품중분류명 == '여성용의류-SPA'"), index = "월", columns = "성별", 
               values = "구매금액", aggfunc = "sum", fill_value = 0).plot.bar(figsize = (8, 6), rot = 0)
plt.title("여성용 SPA의류 월별 총구매액", fontsize = 15)
plt.ylabel("구매액")
plt.xlabel("월")
plt.show()

In [ ]:
# 전체 상품의 시간대별 구매액

vip.groupby("구매시간")["구매금액"].sum().plot(marker = "o", figsize = (16, 12))
plt.xlabel("시간대")
plt.ylabel("총구매액")
plt.title("시간대별 매출액")
plt.show()

In [ ]:
# 식품의 중분류상품별 매출비중

f = vip.query("상품대분류명 == '식품'")
f.groupby("상품중분류명")["구매금액"].sum().plot.pie(autopct = "%1.1f%%", explode = [0.1, 0, 0, 0, 0, 0], figsize = (6, 6))
plt.title("식품의 중분류상품별 매출비중")
plt.ylabel("")
plt.show()

In [ ]:
# 식품의 중분류상품별 거래비중

f = vip.query("상품대분류명 == '식품'")
f.groupby("상품중분류명")["구매금액"].count().plot.pie(autopct = "%1.1f%%", explode = [0.1, 0.2, 0, 0, 0, 0], figsize = (6, 6))
plt.title("식품의 중분류상품별 거래비중")
plt.ylabel("")
plt.show()

In [ ]:
# 식품별 판매단가의 평균

f.groupby("상품중분류명")["판매단가"].mean().plot.bar(rot = 0, color = "skyblue")
plt.title("식품 중분류상품별 판매단가")
plt.show()